In [4]:
from biref_forecasts.biref_fisher_class import FisherBiref
from pspy import so_dict
import numpy as np
from math import pi
from copy import deepcopy
from matplotlib import pyplot as plt
import matplotlib as mpl
import random

mpl.rcParams['lines.linewidth'] = 1.5
mpl.rcParams['lines.markersize'] = 5
mpl.rcParams['lines.markeredgewidth'] = 1.5

# print(mpl.rcParams.keys())

In [6]:
### Planck
args_1s = so_dict.so_dict()
args_1s.read_from_file('paramfiles/Planck_HFI_args.dict')
args_1s['fsky'] = 0.92
args_1s['amp_dust'] = [6. for _ in args_1s['splits']]
args_1a = deepcopy(args_1s)
args_1a['alphas_mapping'] = [0, 0, 0, 0]

args_2s = so_dict.so_dict()
args_2s.read_from_file('paramfiles/Planck_HFI_args.dict')
args_2s['splits'] = [split+s for split in args_1s['splits'] for s in ['A', 'B']]
args_2s['nu_GHzs'] = [freq for freq in args_1s['nu_GHzs'] for s in ['A', 'B']]
args_2s['Nls_filenames'] = [filename for filename in args_1s['Nls_filenames'] for s in ['A', 'B']]
args_2s['Nls_factor'] = 2
args_2s['fsky'] = 0.92
args_2s['amp_dust'] = [6. for _ in args_2s['splits']]
args_2s2 = deepcopy(args_2s)
args_2s2['alphas_mapping'] = [0, 0, 1, 1, 2, 2, 3, 3]
args_2s1a = deepcopy(args_2s)
args_2s1a['alphas_mapping'] = [0, 0, 0, 0, 0, 0, 0, 0]

args_dict = {
    '1split': args_1s,
    '1angle': args_1a,
    '2split': args_2s,
    # '2split2': args_2s2,
    '2split 1angle': args_2s1a,
}

if False:
    for args in args_dict.values():
        args['nu_GHzs'] = [200 for _ in args['splits']]

In [7]:
fisher_dict = {name: FisherBiref(**args).fisher() for name, args in args_dict.items()}
alpha_bias_dict = {name: FisherBiref(**args).bias()['a'] for name, args in args_dict.items()}
beta_bias_dict = {name: FisherBiref(**args).bias()['b'] for name, args in args_dict.items()}

lb = FisherBiref(**args_1s).ls

fisher_dict['1split eff'] = np.block([[np.sum(fisher_dict['1split'][0, 0]) , np.sum(fisher_dict['1split'][0, 1:])],
                                    [np.sum(fisher_dict['1split'][1:, 0]) , np.sum(fisher_dict['1split'][1:, 1:])]])

# prior = np.zeros_like((5, 5))
# prior_index = 4
# prior[prior_index, prior_index] += 0.1**-2 * ((180 / np.pi)**2)
# np.sqrt(1 / np.sum(np.linalg.inv(cov_mat[1:, 1:])))

cov_dict = {name: np.linalg.inv(fisher_mat) * (180 / np.pi)**2 for name, fisher_mat in fisher_dict.items()}

sigmas_dict = {name: np.sqrt(cov_mat.diagonal()) for name, cov_mat in cov_dict.items()}

sigmas_dict

/Users/duparc/Desktop/Stage_M2/codes/biref_forecasts/functions/foregrounds.py:86: RuntimeWarning: divide by zero encountered in power
  return ((ls / 500) ** (alpha + 2)) / fac


{'1split': array([0.11191861, 0.13603795, 0.10349096, 0.09348904, 0.09157124]),
 '1angle': array([0.10442858, 0.09106942]),
 '2split': array([0.11191861, 0.16188698, 0.16188698, 0.11019645, 0.11019645,
        0.09670336, 0.09670336, 0.09274869, 0.09274869]),
 '2split 1angle': array([0.10442858, 0.09106942]),
 '1split eff': array([0.10442858, 0.09106942])}